In [116]:
#Importing libraries

import cv2
import numpy as np
import os
from PIL import Image

In [117]:
#Path for the training images

Path = './Images_face95/'
files = os.listdir(Path)

In [118]:
#Converting each image to Grayscale and appending the data to images

images = []

for name in files:
    for i in range(1,21,4):
        temp = cv2.imread(Path+name+'/'+name+'.'+str(i)+'.jpg')
        temp = cv2.cvtColor(temp,cv2.COLOR_BGR2GRAY)
        temp = cv2.resize(temp, (100,100), interpolation = cv2.INTER_AREA)
        images.append(temp.flatten())
        
# total number of images considered : 360 - 5 of each person
# images converted to grayscale of size : (100,100)

# images : rows - 360
#          columns - 100*100 = 10000
# images.shape : (360,10000)

# temp.shape : (100, 100)

In [119]:
# Calculating the mean of images

images = np.array(images)
image_mean = images.mean(axis = 0)
images = images - image_mean
images = images.T

# transpose of images taken to find eigen vectors of matrix A'A

# image_mean.shape : (100000,)
# images.shape : (10000, 360)

In [120]:
#SVD function
#u is the basis of eigen faces

u,s,v = np.linalg.svd(images, full_matrices=False)

# u.shape : (10000, 360)
# There are 360 eigen faces currently

In [121]:
#Read the imput images for face recognition

test = np.array(cv2.imread('./Input/test_image.jpg'))
test = cv2.cvtColor(test,cv2.COLOR_BGR2GRAY)
test = cv2.resize(test, (100,100), interpolation = cv2.INTER_AREA)

test_img = test.flatten()

# test_img.shape : (10000, )

In [122]:
#Substracting mean

img = test_img - image_mean
test_img = img.T

In [123]:
#Dot product of all the images and U matrix to find the projection co-efficients

dot_train = np.empty(shape = (u.shape[1], u.shape[1]),  dtype=np.int8)
temp = np.empty(shape = (1, u.shape[1]),  dtype=np.int8)

# dot_train.shape : (360, 360)
# temp.shape : (1, 360)
# images.shape : (10000, 360)

for i in range(images.shape[1]):    
    for c in range(u.shape[1]):    
        temp[0,c] = np.sum(images[:,i] * u[:,c]
        
    dot_train[:, i] = temp[:]
                           
# got this random error now, even i'm confused why its happening

SyntaxError: invalid syntax (<ipython-input-123-58ffef9f3801>, line 14)

In [78]:
#Dot product of test image and U matrix

test_img = np.array(test_img)
dot_test = np.empty(shape = (1, u.shape[1]), dtype=np.int8)   ##changes

# dot_test.shape : (10000, 360)
# u.shape : (10000, 360)
# test_img.shape : (10000, )

for col in range(u.shape[1]):    
    dot_test[0,col] = np.sum(test_img[:] * u[:,col])


In [92]:
# calculating the variation between input image and test images

sub = np.empty(shape = (u.shape[1], u.shape[1]), dtype=np.int8)

for col in range(u.shape[1]):
    sub[:,col] = dot_train[:,col] - dot_test[:]

ValueError: could not broadcast input array from shape (3600000) into shape (360)

In [80]:
#Finding norm of all the colums
answer = np.empty(shape=(u.shape[1],))

for c in range(sub.shape[1]):    
    answer[c] = np.linalg.norm(sub[:,c])

In [81]:
# FACE RECOGNITION

temp_ans = np.empty(shape=(u.shape[1],))
temp=np.copy(answer)
temp.sort()
check = temp[0]

index=0
for i in range(answer.shape[0]):
    if check == answer[i]:
        index=i
        break

In [82]:
# Checking for corresponding image for minimum answer

i = 0
flag = 0
for name in files:
    for j in range(1,21,4):
        if index == i:
            print( "The predicted face is: ",name)
            flag = 1
            break
        
        else:
            i=i+1
    if flag:
        break

The predicted face is:  pjrand


In [69]:
"""
# prints eigen faces

face300 = np.array(u.T[3])
print(face300.shape)

min = np.min(face300)
max = np.max(face300)
print(min,max)

face_eigen = (face300 - min)*255/(max - min)
print(face_eigen.shape)
face_eigen_reshape = np.reshape(face_eigen, (100, 100))
Image.fromarray(face_eigen_reshape).show()
"""

In [ ]:
#IGNORE FROM HERE

In [90]:
## dont run now!

#Dot product of test image and U matrix

test_img = np.array(test_img)
test_x = np.empty(shape = (u.shape[0], u.shape[1]),  dtype=np.int8)

# test_x.shape : (10000, 360)
# u.shape : (10000, 360)
# test_img.shape : (10000, )

for col in range(u.shape[1]):    
    test_x[:,col] = test_img[:] * u[:,col]

dot_test = np.array(test_x, dtype='int8').flatten()

In [91]:
## havent run yet

#Dot product of all the images and U matrix to find the projection co-efficients

dot_train = np.empty(shape = (u.shape[0]*u.shape[1], u.shape[1]),  dtype=np.int8)
temp = np.empty(shape = (u.shape[0], u.shape[1]),  dtype=np.int8)

# dot_train.shape : (360 0000, 360)
# temp.shape : (10000, 360)
# images.shape : (10000, 360)

for i in range(images.shape[1]):    
    for c in range(u.shape[1]):    
        temp[:,c] = images[:,i] * u[:,c]
        
    temp_2 = np.array(temp, dtype='int8').flatten()
    dot_train[:, i] = temp_2[:]

In [95]:
# calculating the variation between input image and test images

sub = np.empty(shape = (u.shape[1]*u.shape[0], u.shape[1]), dtype=np.int8)

for col in range(u.shape[1]):
    sub[:,col] = dot_train[:,col] - dot_test[:]

MemoryError: Unable to allocate array with shape (3600000, 360) and data type int8

In [96]:
#Finding norm of all the colums
answer = np.empty(shape=(u.shape[1],))

for c in range(sub.shape[1]):    
    answer[c] = np.linalg.norm(sub[:,c])

In [81]:
# FACE RECOGNITION

temp_ans = np.empty(shape=(u.shape[1],))
temp=np.copy(answer)
temp.sort()
check = temp[0]

index=0
for i in range(answer.shape[0]):
    if check == answer[i]:
        index=i
        break

In [82]:
# Checking for corresponding image for minimum answer

i = 0
flag = 0
for name in files:
    for j in range(1,21,4):
        if index == i:
            print( "The predicted face is: ",name)
            flag = 1
            break
        
        else:
            i=i+1
    if flag:
        break

The predicted face is:  pjrand
